# Sense HAT for PYNQ:Level meter

This notebook illustrates how to implement a level meter use the [Sense HAT](https://www.raspberrypi.org/products/sense-hat/) . It reads the acceleration data from Sense HAT's IMU(Inertial Measurement Unit) sensor and display a 2x2 matrix on LED matrix to show current level status.

This example notebook does the following.
1. import python libraries
2. select rpi switch and using MicroblazeLibrary
3. configure the I2C device
4. read single IMU data
5. implement a level meter

![PYNQ with Sense HAT](data/PYNQ_with_Sense_HAT.jpg)


### 1. Sense HAT Introduction

The Sense HAT, which is a fundamental part of the [Astro Pi](https://astro-pi.org/) mission, allows your board to sense the world around it.I thas an 8×8 RGB LED matrix, a five-button joystick and includes the following sensors:

* Gyroscope
* Accelerometer
* Magnetometer
* Temperature
* Barometric pressure
* Humidity

![Sense HAT add-on board](data/Sense_HAT.jpg)


### 2. Prepare the overlay
Download the overlay first, then select the shared pin to be connected to
RPI header (by default, the pins will be connected to PMODA instead).

In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
from imp import reload
from time import sleep
import numpy as np
from sensehat import *

In [2]:
base = BaseOverlay('base.bit')
base.select_rpi()
lib = MicroblazeLibrary(base.RPI, ['i2c','xio_switch','circular_buffer'])

### 3. Configure the I2C device
Initialize the I2C device and set the I2C pin of RPI header 

In [3]:
i2c = lib.i2c_open_device(1)
lib.set_pin(2, lib.SDA1)
lib.set_pin(3, lib.SCL1)

### 4. Define level_meter function
the level_meter function get the acceleration data and display current level status

In [4]:
def level_meter(accel_x, accel_y, buf):
    center_data = np.array(
        [
       #[1, 2, 3, 4, 5, 6, 7, 8],
        [0, 0, 0, 0, 0, 0, 0, 0],# 1
        [0, 0, 0, 0, 0, 0, 0, 0],# 2
        [0, 0, 0, 0, 0, 0, 0, 0],# 3
        [0, 0, 0, 1, 1, 0, 0, 0],# 4
        [0, 0, 0, 1, 1, 0, 0, 0],# 5
        [0, 0, 0, 0, 0, 0, 0, 0],# 6
        [0, 0, 0, 0, 0, 0, 0, 0],# 7
        [0, 0, 0, 0, 0, 0, 0, 0] # 8
        ]
    )
    buf[0] = 0
    red_color = 0
    green_color = 0
    blue_color = 50
    x_value = int(accel_x * 1.5)
    y_value = int(accel_y * 1.5)
    x_value = 3 if x_value < -3 else 5 if x_value > 3 else abs(x_value) if x_value < 0 else 8 - x_value
    y_value = 3 if y_value < -3 else 5 if y_value > 3 else abs(y_value) if y_value < 0 else 8 - y_value
    frame_buffer = np.hstack((center_data[:,x_value:8],center_data[:,0:x_value]))
    frame_buffer = np.vstack((frame_buffer[y_value:8],frame_buffer[0:y_value]))
    frame_buffer = np.rot90(frame_buffer, 2)
    for y in range(0,8) :
        for x in range(0,8) :
            buf[1+x+8*0+3*8*y] = red_color
            buf[1+x+8*1+3*8*y] = green_color
            buf[1+x+8*2+3*8*y] = frame_buffer[y][x]*blue_color

### 5. Read single IMU data
The IMU sensor of Sense HAT is LSM9DS1 

In [5]:
sensor = adafruit_lsm9ds1.LSM9DS1_I2C(i2c)
accel_x, accel_y, accel_z = sensor.acceleration
mag_x, mag_y, mag_z = sensor.magnetic
gyro_x, gyro_y, gyro_z = sensor.gyro
temp = sensor.temperature
# Print values.
print('Acceleration (m/s^2): ({0:0.3f},{1:0.3f},{2:0.3f})'.format(accel_x, accel_y, accel_z))
print('Magnetometer (gauss): ({0:0.3f},{1:0.3f},{2:0.3f})'.format(mag_x, mag_y, mag_z))
print('Gyroscope (degrees/sec): ({0:0.3f},{1:0.3f},{2:0.3f})'.format(gyro_x, gyro_y, gyro_z))
print('Temperature: {0:0.3f}C'.format(temp))

Acceleration (m/s^2): (-0.083,-0.074,9.884)
Magnetometer (gauss): (0.270,0.381,-0.463)
Gyroscope (degrees/sec): (5.845,4.043,3.754)
Temperature: 22.500C


### 6.Implement a level meter
Press Run to implement a level meter, press BTN0 on PYNQ Z2 board to exit.

In [6]:
buf = bytearray(193)
buf[0] = 0

In [7]:
while not base.buttons[0].read():
    accel_x, accel_y, accel_z = sensor.acceleration
    level_meter(accel_x, accel_y, buf)
    i2c.write(0x46, buf, 193)
    sleep(0.2)
level_meter(0, 0, buf)
i2c.write(0x46, buf, 193)

### 7.Clean up

In [8]:
i2c.close()
base.select_pmoda()